In [845]:
# Import dependencies
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.tree import DecisionTreeRegressor

# Feature Ingineering

In [846]:
# Read games, population, and gdp data from sql
db_path = '../project-4/games_data.sqlite'
conn = sqlite3.connect(db_path)
games = pd.read_sql_query("SELECT * FROM data", conn)
gdp = pd.read_sql_query("SELECT * FROM gdp", conn)
population = pd.read_sql_query("SELECT * FROM population", conn)

In [847]:
# Show the games table
games.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [848]:
# Show the GDP table
gdp.head()

,Year,Total
0,1960,9.940132e+12
1,1961,1.043138e+13
2,1962,1.110791e+13
3,1963,1.202426e+13
4,1964,1.327069e+13


In [849]:
# Show population table
population.head()

,Year,Total
0,1960,56.042872
1,1961,55.875365
2,1962,55.738212
3,1963,55.608248
4,1964,55.497390


In [850]:
# Rename Total column in population
population.rename(columns={'Total': 'AVG_15-64_Pop%'}, inplace=True)

In [851]:
# Join games and population tables on year
games = games.merge(population, on = ["Year"], how="left")

In [852]:
# Rename Total column in GDP table
gdp.rename(columns={'Total': 'World_GDP'}, inplace=True)

In [853]:
# Join games and gdp tables on year
games = games.merge(gdp, on = ["Year"], how="left")

In [854]:
# Take a log of global sales, gdp and population % to eliminate extreme values
games["Global_Sales"] = np.log(games["Global_Sales"])
games["World_GDP"] = np.log(games["World_GDP"])
games["AVG_15-64_Pop%"] = np.log(games["AVG_15-64_Pop%"])

In [855]:
# We drop Rank, NA_Sales, EU_Sales, JP_Sales, Other_Sales to avoid multicolinearity issues,
# as Global_Sales is a summation of other sales columns and Rank is determined by Global_Sales
games.drop(columns=['Rank', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'], inplace=True)
games.head()

,Name,Platform,Year,Genre,Publisher,Global_Sales,AVG_15-64_Pop%,World_GDP
0,Wii Sports,Wii,2006,Sports,Nintendo,4.415703,4.140218,33.610978
1,Super Mario Bros.,NES,1985,Platform,Nintendo,3.694862,4.068963,32.179648
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,3.578506,4.148259,33.855237
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,3.496508,4.151141,33.804465
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,3.445852,4.093306,33.095176


In [856]:
# Creating an previous year avarege sales by platform column 
games_avg_pltf = games.groupby(by = ["Platform", "Year"], as_index=False)["Global_Sales"].mean()
games_avg_pltf["Year"]=games_avg_pltf["Year"] + 1 
games = games.merge(games_avg_pltf, on = ["Platform", "Year"], how= "left")
games = games.rename(columns={"Global_Sales_x": "Global_Sales", "Global_Sales_y":"Previous_Year_Platform_AVG_Sales"})
games.head()

,Name,Platform,Year,Genre,Publisher,Global_Sales,AVG_15-64_Pop%,World_GDP,Previous_Year_Platform_AVG_Sales
0,Wii Sports,Wii,2006,Sports,Nintendo,4.415703,4.140218,33.610978,NaN
1,Super Mario Bros.,NES,1985,Platform,Nintendo,3.694862,4.068963,32.179648,0.670776
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,3.578506,4.148259,33.855237,-1.540902
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,3.496508,4.151141,33.804465,-1.510655
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,3.445852,4.093306,33.095176,-0.490695


In [857]:
# Creating an previous year avarege sales by Publisher column 
games_avg_pub = games.groupby(by = ["Publisher", "Year"], as_index=False)["Global_Sales"].mean()
games_avg_pub["Year"]=games_avg_pub["Year"] + 1 
games = games.merge(games_avg_pub, on = ["Publisher", "Year"], how= "left")
games = games.rename(columns={"Global_Sales_x": "Global_Sales", "Global_Sales_y":"Previous_Year_Publisher_AVG_Sales"})
games.head()

,Name,Platform,Year,Genre,Publisher,Global_Sales,AVG_15-64_Pop%,World_GDP,Previous_Year_Platform_AVG_Sales,Previous_Year_Publisher_AVG_Sales
0,Wii Sports,Wii,2006,Sports,Nintendo,4.415703,4.140218,33.610978,NaN,-0.416521
1,Super Mario Bros.,NES,1985,Platform,Nintendo,3.694862,4.068963,32.179648,0.670776,0.930919
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,3.578506,4.148259,33.855237,-1.540902,-0.594387
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,3.496508,4.151141,33.804465,-1.510655,-0.351708
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,3.445852,4.093306,33.095176,-0.490695,-0.294144


In [858]:
# Creating an previous year avarege sales by Genre column 
games_avg_genre = games.groupby(by = ["Genre", "Year"], as_index=False)["Global_Sales"].mean()
games_avg_genre["Year"]=games_avg_genre["Year"] + 1 
games = games.merge(games_avg_genre, on = ["Genre", "Year"], how= "left")
games = games.rename(columns={"Global_Sales_x": "Global_Sales", "Global_Sales_y":"Previous_Year_Genre_AVG_Sales"})
games.head()

,Name,Platform,Year,Genre,Publisher,Global_Sales,AVG_15-64_Pop%,World_GDP,Previous_Year_Platform_AVG_Sales,Previous_Year_Publisher_AVG_Sales,Previous_Year_Genre_AVG_Sales
0,Wii Sports,Wii,2006,Sports,Nintendo,4.415703,4.140218,33.610978,NaN,-0.416521,-1.495709
1,Super Mario Bros.,NES,1985,Platform,Nintendo,3.694862,4.068963,32.179648,0.670776,0.930919,-0.371064
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,3.578506,4.148259,33.855237,-1.540902,-0.594387,-1.894421
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,3.496508,4.151141,33.804465,-1.510655,-0.351708,-1.503771
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,3.445852,4.093306,33.095176,-0.490695,-0.294144,-1.095763


In [859]:
# Determine the number of unique values in each column.
games.nunique()

Name                                 11493
Platform                                31
Year                                    39
Genre                                   12
Publisher                              578
Global_Sales                           623
AVG_15-64_Pop%                          39
World_GDP                               39
Previous_Year_Platform_AVG_Sales       202
Previous_Year_Publisher_AVG_Sales     1074
Previous_Year_Genre_AVG_Sales          356
dtype: int64

In [860]:
# Dropping Year and categorical columns
games.drop(columns=['Year','Genre','Publisher', 'Platform'], inplace=True)

In [861]:
# Displaying the table
games.head()

,Name,Global_Sales,AVG_15-64_Pop%,World_GDP,Previous_Year_Platform_AVG_Sales,Previous_Year_Publisher_AVG_Sales,Previous_Year_Genre_AVG_Sales
0,Wii Sports,4.415703,4.140218,33.610978,NaN,-0.416521,-1.495709
1,Super Mario Bros.,3.694862,4.068963,32.179648,0.670776,0.930919,-0.371064
2,Mario Kart Wii,3.578506,4.148259,33.855237,-1.540902,-0.594387,-1.894421
3,Wii Sports Resort,3.496508,4.151141,33.804465,-1.510655,-0.351708,-1.503771
4,Pokemon Red/Pokemon Blue,3.445852,4.093306,33.095176,-0.490695,-0.294144,-1.095763


In [862]:
# Splitting the names to words and collecting words that have more than 4 letters
words = games['Name'].str.split().explode()
filtered_words = words[words.str.len() >= 4]

In [863]:
# Creating a dataframe with the freaquency of used words for games
from collections import Counter
word_counts = Counter(filtered_words)
word_counts_df = pd.DataFrame.from_dict(word_counts, orient='index', columns=['Frequency'])
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)
word_counts_df.head()

,Frequency
World,386
Game,300
Super,289
Star,235
Soccer,221


In [864]:
# Creating the list with top 5% of the most used words
top_5_percent_threshold = np.percentile(word_counts_df['Frequency'], 95)
top_5_percent_words = word_counts_df[word_counts_df['Frequency'] >= top_5_percent_threshold]
words_list =list(top_5_percent_words.index)

In [865]:
def world_counter(game_name):
    
    # Split the game name into a list of words
    words = game_name.split()
    
    # Initialize a counter to keep track of how many words match the words in words_list
    counter = 0

    # Loop through each word in the list of words
    for w in words:

        # Check if the word is present in the predefined list called words_list
        if w in words_list:
             # If the word is found in words_list, increment the counter by 1
            counter +=1
    return counter 

In [866]:
# Add a "Word_Counter" column by applying the world_counter function to the 'Name' column for each row
games["Word_Counter"] = games.apply(lambda x: world_counter(x['Name']), axis =1)

In [867]:
# # Creating  avarege sales by Word_Counter column 
word_sales_mean = games.groupby('Word_Counter')['Global_Sales'].mean()
games['Word_Counter_Sales'] = games['Word_Counter'].map(word_sales_mean)

In [868]:
# Displaying the dataset
games.head()

,Name,Global_Sales,AVG_15-64_Pop%,World_GDP,Previous_Year_Platform_AVG_Sales,Previous_Year_Publisher_AVG_Sales,Previous_Year_Genre_AVG_Sales,Word_Counter,Word_Counter_Sales
0,Wii Sports,4.415703,4.140218,33.610978,NaN,-0.416521,-1.495709,1,-1.819978
1,Super Mario Bros.,3.694862,4.068963,32.179648,0.670776,0.930919,-0.371064,3,-1.448954
2,Mario Kart Wii,3.578506,4.148259,33.855237,-1.540902,-0.594387,-1.894421,2,-1.543546
3,Wii Sports Resort,3.496508,4.151141,33.804465,-1.510655,-0.351708,-1.503771,1,-1.819978
4,Pokemon Red/Pokemon Blue,3.445852,4.093306,33.095176,-0.490695,-0.294144,-1.095763,2,-1.543546


In [869]:
# Dropping Name column
games.drop(columns =['Name'], inplace = True, axis =1)

In [870]:
# Filling NAs with means 
games["Previous_Year_Publisher_AVG_Sales"].fillna(games["Previous_Year_Publisher_AVG_Sales"].mean(), inplace=True)
games["Previous_Year_Platform_AVG_Sales"].fillna(games["Previous_Year_Platform_AVG_Sales"].mean(), inplace=True)
games["Previous_Year_Genre_AVG_Sales"].fillna(games["Previous_Year_Genre_AVG_Sales"].mean(), inplace=True)
games["Word_Counter_Sales"].fillna(games["Word_Counter_Sales"].mean(), inplace=True)

In [871]:
# Displaying datatable
games.head()

,Global_Sales,AVG_15-64_Pop%,World_GDP,Previous_Year_Platform_AVG_Sales,Previous_Year_Publisher_AVG_Sales,Previous_Year_Genre_AVG_Sales,Word_Counter,Word_Counter_Sales
0,4.415703,4.140218,33.610978,-1.622911,-0.416521,-1.495709,1,-1.819978
1,3.694862,4.068963,32.179648,0.670776,0.930919,-0.371064,3,-1.448954
2,3.578506,4.148259,33.855237,-1.540902,-0.594387,-1.894421,2,-1.543546
3,3.496508,4.151141,33.804465,-1.510655,-0.351708,-1.503771,1,-1.819978
4,3.445852,4.093306,33.095176,-0.490695,-0.294144,-1.095763,2,-1.543546


# Modelling

## Linear Regression Model

In [819]:
# Defining the target and features columns
target_column = 'Global_Sales'
feature_columns = games.columns.drop(target_column)

In [820]:
# Splitting the data into features and target
X = games[feature_columns]
y = games[target_column]

In [821]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [823]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [824]:
# Creating the linear regression model
model_lr = LinearRegression()

In [825]:
# Training the model
model_lr.fit(X_train_scaled, y_train)

LinearRegression()

In [826]:
# Making predictions on the test set
y_pred = model_lr.predict(X_test_scaled)

In [827]:
# Evaluating the model and printing the reuslts
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 1.5896156881958647
R-squared: 0.2534117261931119


## Decision Tree Model

In [829]:
# Creating the decision tree model
model_dt = DecisionTreeRegressor(random_state=42)

In [830]:
# Training the model
model_dt.fit(X_train_scaled, y_train)

DecisionTreeRegressor(random_state=42)

In [831]:
# Making predictions on the test set
y_pred = model_dt.predict(X_test_scaled)

In [832]:
# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2.346470871647009
R-squared: -0.10205734040618641


## Neural Network Model

In [835]:
# Create a Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Choice of activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Choice of number of neurons in first layer
    nn_model.add(tf.keras.layers.InputLayer(shape=(7,)))
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=20,
        step=2), activation=activation))

    # Choice of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=14,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="mse", optimizer='adam', metrics=["mse"])

    return nn_model

In [836]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="mse",
    max_epochs=30,
    hyperband_iterations=2)

In [837]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=30,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 01m 13s]
mse: 5.185958385467529

Best mse So Far: 5.182490348815918
Total elapsed time: 01h 11m 18s


In [838]:
# Getting the best model
best_model = tuner.get_best_models(1)[0]

c:\Users\ulyan\anaconda3\Lib\site-packages\keras_tuner\src\tuners\hyperband.py:435: UserWarning: Model 'sequential' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  model.build_from_config(
c:\Users\ulyan\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [839]:
# Evaluate the model on the test set
model_mse = best_model.evaluate(X_test_scaled, y_test, verbose=2)

# Print the MSE
print(f"Mean Squared Error: {model_mse}")

104/104 - 1s - 11ms/step - loss: 5.0822 - mse: 5.0822
Mean Squared Error: [5.082244396209717, 5.082244396209717]
